In [ ]:
print("\n" + "="*70)
print("MODEL TRAINING SUMMARY")
print("="*70)

print(f"\n1. Dataset Information:")
print(f"   - Total samples: {len(df):,}")
print(f"   - Number of features: {X.shape[1]}")
print(f"   - Training set: {len(X_train):,} samples ({100*len(X_train)/len(df):.0f}%)")
print(f"   - Test set: {len(X_test):,} samples ({100*len(X_test)/len(df):.0f}%)")

print(f"\n2. Models Trained: {len(trained_models)}")
for name in trained_models.keys():
    print(f"   - {name}")

print(f"\n3. Best Model: {best_model_name}")
print(f"   - Test R² Score: {best_results['Test R²']:.4f}")
print(f"   - Test RMSE: {best_results['Test RMSE']:.4f}")
print(f"   - Test MAE: {best_results['Test MAE']:.4f}")
print(f"   - Cross-Val R²: {cv_df[cv_df['Model'] == best_model_name]['CV Mean R²'].values[0]:.4f}")

print(f"\n4. Top 5 Performing Models (by Test R²):")
top_models = results_df.nlargest(5, 'Test R²')
for i, (idx, row) in enumerate(top_models.iterrows(), 1):
    print(f"   {i}. {row['Model']:20s} - Test R²: {row['Test R²']:.4f}")

print(f"\n5. Model Improvements:")
print(f"   - Best Test RMSE: {results_df['Test RMSE'].min():.4f}")
print(f"   - Best Test MAE: {results_df['Test MAE'].min():.4f}")
print(f"   - Baseline (Linear Regression) R²: {results_df[results_df['Model'] == 'Linear Regression']['Test R²'].values[0]:.4f}")
print(f"   - Best improvement: {best_results['Test R²'] - results_df[results_df['Model'] == 'Linear Regression']['Test R²'].values[0]:.4f}")

print(f"\n6. Model Artifacts Saved:")
print(f"   - Models: {len(list(model_dir.glob('*.pkl'))) - 1} (+ scaler)")
print(f"   - Results summary: model_comparison_results.csv")

print("\n" + "="*70)

## 9. Training Summary

In [ ]:
# Save models
from pathlib import Path

model_dir = Path('models')
model_dir.mkdir(exist_ok=True)

for model_name, model in trained_models.items():
    model_path = model_dir / f"{model_name.replace(' ', '_').lower()}.pkl"
    joblib.dump(model, model_path)
    print(f"✓ Saved {model_name} to {model_path}")

# Save scaler
scaler_path = model_dir / "scaler.pkl"
joblib.dump(scaler, scaler_path)
print(f"✓ Saved scaler to {scaler_path}")

# Save results summary
results_df.to_csv(model_dir / 'model_comparison_results.csv', index=False)
print(f"✓ Saved results summary to {model_dir / 'model_comparison_results.csv'}")

print(f"\nAll models saved to: {model_dir.absolute()}")

## 8. Model Persistence

In [ ]:
# 5-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = []

print("Performing 5-Fold Cross-Validation...\n")

for model_name, model in trained_models.items():
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=kfold, scoring='r2', n_jobs=-1)
    cv_results.append({
        'Model': model_name,
        'CV Mean R²': cv_scores.mean(),
        'CV Std': cv_scores.std(),
        'CV Scores': cv_scores
    })
    print(f"{model_name:20s}: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

cv_df = pd.DataFrame(cv_results)

# Visualize CV results
fig, ax = plt.subplots(figsize=(12, 6))

cv_means = [r['CV Mean R²'] for r in cv_results]
cv_stds = [r['CV Std'] for r in cv_results]
model_names = [r['Model'] for r in cv_results]

x = np.arange(len(model_names))
ax.bar(x, cv_means, yerr=cv_stds, capsize=5, alpha=0.8)
ax.set_ylabel('R² Score')
ax.set_title('5-Fold Cross-Validation Results')
ax.set_xticks(x)
ax.set_xticklabels(model_names, rotation=45, ha='right')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7. Cross-Validation Analysis

In [ ]:
# Feature importance for tree-based models
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

tree_models = ['Random Forest', 'Gradient Boosting', 'XGBoost']

for idx, model_name in enumerate(tree_models):
    model = trained_models[model_name]
    
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        indices = np.argsort(importances)[-15:]  # Top 15
        
        ax = axes[idx]
        ax.barh(np.array(X.columns)[indices], importances[indices])
        ax.set_xlabel('Importance Score')
        ax.set_title(f'{model_name} - Top 15 Features')
        ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 6. Feature Importance (Tree-based Models)

In [ ]:
# Select best model
best_model_name = results_df.loc[results_df['Test R²'].idxmax(), 'Model']
best_model = trained_models[best_model_name]
best_results = results_df[results_df['Model'] == best_model_name].iloc[0]

print(f"\n{'='*60}")
print(f"BEST MODEL: {best_model_name}")
print(f"{'='*60}")
print(f"Test R² Score:  {best_results['Test R²']:.4f}")
print(f"Test RMSE:      {best_results['Test RMSE']:.4f}")
print(f"Test MAE:       {best_results['Test MAE']:.4f}")
print(f"Train R² Score: {best_results['Train R²']:.4f}")
print(f"Overfitting Gap: {(best_results['Train R²'] - best_results['Test R²']):.4f}")
print(f"{'='*60}\n")

# Get predictions from best model
y_test_pred_best = best_model.predict(X_test_scaled)

# Residuals
residuals = y_test - y_test_pred_best

# Create analysis plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Actual vs Predicted
ax = axes[0, 0]
ax.scatter(y_test, y_test_pred_best, alpha=0.6)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax.set_xlabel('Actual Risk Score')
ax.set_ylabel('Predicted Risk Score')
ax.set_title(f'{best_model_name}: Actual vs Predicted')
ax.grid(True, alpha=0.3)

# 2. Residuals plot
ax = axes[0, 1]
ax.scatter(y_test_pred_best, residuals, alpha=0.6)
ax.axhline(y=0, color='r', linestyle='--', lw=2)
ax.set_xlabel('Predicted Risk Score')
ax.set_ylabel('Residuals')
ax.set_title('Residual Plot')
ax.grid(True, alpha=0.3)

# 3. Residuals distribution
ax = axes[1, 0]
ax.hist(residuals, bins=30, alpha=0.7, edgecolor='black')
ax.axvline(residuals.mean(), color='r', linestyle='--', label=f'Mean: {residuals.mean():.4f}')
ax.set_xlabel('Residual Value')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Residuals')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# 4. Q-Q plot
from scipy import stats
ax = axes[1, 1]
stats.probplot(residuals, dist="norm", plot=ax)
ax.set_title('Q-Q Plot (Normality Check)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Best Model Analysis

In [ ]:
# R² Comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. R² Score Comparison
ax = axes[0, 0]
x = np.arange(len(results_df))
width = 0.35
ax.bar(x - width/2, results_df['Train R²'], width, label='Train', alpha=0.8)
ax.bar(x + width/2, results_df['Test R²'], width, label='Test', alpha=0.8)
ax.set_ylabel('R² Score')
ax.set_title('R² Score Comparison (Train vs Test)')
ax.set_xticks(x)
ax.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. RMSE Comparison
ax = axes[0, 1]
ax.bar(x - width/2, results_df['Train RMSE'], width, label='Train', alpha=0.8)
ax.bar(x + width/2, results_df['Test RMSE'], width, label='Test', alpha=0.8)
ax.set_ylabel('RMSE')
ax.set_title('RMSE Comparison (Train vs Test)')
ax.set_xticks(x)
ax.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. MAE Comparison
ax = axes[1, 0]
ax.bar(x - width/2, results_df['Train MAE'], width, label='Train', alpha=0.8)
ax.bar(x + width/2, results_df['Test MAE'], width, label='Test', alpha=0.8)
ax.set_ylabel('MAE')
ax.set_title('MAE Comparison (Train vs Test)')
ax.set_xticks(x)
ax.set_xticklabels(results_df['Model'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. Test R² Ranking
ax = axes[1, 1]
sorted_results = results_df.sort_values('Test R²', ascending=True)
colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(sorted_results)))
ax.barh(sorted_results['Model'], sorted_results['Test R²'], color=colors)
ax.set_xlabel('Test R² Score')
ax.set_title('Model Performance Ranking (by Test R²)')
ax.grid(True, alpha=0.3, axis='x')

for i, v in enumerate(sorted_results['Test R²']):
    ax.text(v + 0.01, i, f'{v:.3f}', va='center')

plt.tight_layout()
plt.show()

## 4. Model Comparison Visualizations

In [ ]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=0.01, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1, verbosity=0)
}

# Train all models
trained_models = {}
training_results = []

print("Training models...\n")
for name, model in models.items():
    print(f"Training {name}...", end=" ")
    model.fit(X_train_scaled, y_train)
    trained_models[name] = model
    
    # Predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    training_results.append({
        'Model': name,
        'Train R²': train_r2,
        'Test R²': test_r2,
        'Train RMSE': train_rmse,
        'Test RMSE': test_rmse,
        'Train MAE': train_mae,
        'Test MAE': test_mae
    })
    
    print(f"✓ (Train R²: {train_r2:.3f}, Test R²: {test_r2:.3f})")

results_df = pd.DataFrame(training_results)
print("\n" + "="*80)
print(results_df.to_string(index=False))
print("="*80)

## 3. Train Multiple Models

In [ ]:
# Prepare features and target
feature_cols = [col for col in df.columns if col != 'risk_score']
X = df[feature_cols].copy()
y = df['risk_score'].copy()

# Handle missing values
X = X.replace([np.inf, -np.inf], np.nan).fillna(0)

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to DataFrame for feature names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

print(f"Training set: {X_train_scaled.shape}")
print(f"Test set: {X_test_scaled.shape}")
print(f"Train-test split: {len(X_train)}/{len(X_test)} ({100*len(X_train)/len(X):.0f}% / {100*len(X_test)/len(X):.0f}%)")

## 2. Train-Test Split and Scaling

In [ ]:
# Generate data (same as feature engineering)
np.random.seed(42)
n_samples = 1000

lat_range = (15.6, 22.0)
lon_range = (72.6, 80.9)

data = {
    'latitude': np.random.uniform(lat_range[0], lat_range[1], n_samples),
    'longitude': np.random.uniform(lon_range[0], lon_range[1], n_samples),
    'temperature': np.random.normal(25, 5, n_samples),
    'precipitation': np.random.exponential(2, n_samples),
    'humidity': np.random.normal(60, 15, n_samples),
    'wind_speed': np.random.exponential(3, n_samples),
    'forest_cover': np.random.uniform(0, 1, n_samples),
    'agricultural_area': np.random.uniform(0, 1, n_samples),
    'urban_area': np.random.uniform(0, 1, n_samples),
    'water_bodies': np.random.uniform(0, 0.3, n_samples),
    'species_count': np.random.poisson(15, n_samples),
    'endemic_species': np.random.poisson(2, n_samples),
    'threatened_species': np.random.poisson(1, n_samples),
    'elevation': np.random.normal(500, 300, n_samples),
    'population_density': np.random.exponential(100, n_samples)
}

df = pd.DataFrame(data)

# Generate risk score
risk_score = (
    0.3 * (1 - df['forest_cover']) +
    0.2 * df['urban_area'] +
    0.15 * np.abs(df['temperature'] - 25) / 10 +
    0.1 * (1 / (df['species_count'] + 1)) +
    0.1 * df['population_density'] / 1000 +
    0.15 * np.random.normal(0, 0.1, n_samples)
)

df['risk_score'] = np.clip(risk_score, 0, 1)

# Feature engineering (simplified from notebook 2)
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat/2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Spatial features
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()
df['distance_from_center'] = df.apply(
    lambda row: haversine_distance(row['latitude'], row['longitude'], center_lat, center_lon), axis=1
)

# Interaction features
df['biodiversity_index'] = df['species_count'] * 0.5 + df['endemic_species'] * 0.3 + df['threatened_species'] * 0.2
df['threat_index'] = (
    (1 - df['forest_cover']) * 0.3 + 
    df['urban_area'] * 0.25 + 
    df['population_density'] / df['population_density'].max() * 0.25
)
df['habitat_quality'] = np.clip(df['forest_cover'] * 0.5 + (1 - df['urban_area']) * 0.3, 0, 1)

# Interaction terms
df['temp_forest_interaction'] = df['temperature'] * df['forest_cover']
df['urban_pop_interaction'] = df['urban_area'] * df['population_density']

print(f"Dataset prepared: {df.shape}")
print(f"Target variable (risk_score) statistics:")
print(f"  Mean: {df['risk_score'].mean():.3f}")
print(f"  Std: {df['risk_score'].std():.3f}")
print(f"  Min: {df['risk_score'].min():.3f}")
print(f"  Max: {df['risk_score'].max():.3f}")

## 1. Prepare Training Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# ML Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import joblib

# Visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

# EcoPredict Model Training

Train and compare multiple machine learning models for ecological risk prediction.